In [1]:
import pandas as pd
import torch as T
import torch.nn as nn
import torch.nn.functional as ff
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import pickle

import matplotlib.pyplot as plt

from dataclasses import dataclass
import json
from tqdm import tqdm
import numpy as np
import os
from typing import Sequence
from datetime import datetime, timedelta


pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)

# **Look up**

In [8]:
data_turn_of = pd.read_csv('../main_datasets/processed_datasets/6. Плановые-Внеплановые отключения 01.10.2023-30.04.2023.csv')

In [9]:
data_work_heat_station = pd.read_csv('../main_datasets/processed_datasets/6. Плановые-Внеплановые отключения 01.10.2023-30.04.2023.csv')

In [ ]:
data_work_heat_station

In [11]:
data_turn_of['Вид отключения'].value_counts()

Вид отключения
Плановое       107
Внеплановое      8
Name: count, dtype: int64

In [15]:
data_turn_of[data_turn_of['Вид отключения'] == 'Внеплановое']

,Причина,Источник,Дата регистрации отключения,Планируемая дата отключения,Планируемая дата включения,Фактическая дата отключения,Фактическая дата включения,Вид отключения,УНОМ,Адрес
34,Определение утечки,MOEK,2023-11-03 14:02:45.000000,NaN,2023-11-03 12:00:00.000000,2023-11-03 06:00:00.000000,2023-11-03 12:00:00.000000,Внеплановое,18679,внутригородская территория муниципальный округ...
35,Определение утечки,MOEK,2023-11-03 14:02:45.000000,NaN,2023-11-03 12:00:00.000000,2023-11-03 06:00:00.000000,2023-11-03 12:00:00.000000,Внеплановое,18678,внутригородская территория муниципальный округ...
36,Определение утечки,MOEK,2023-11-03 14:02:45.000000,NaN,2023-11-03 12:00:00.000000,2023-11-03 06:00:00.000000,2023-11-03 12:00:00.000000,Внеплановое,18680,внутригородская территория муниципальный округ...
69,Ремонт участка трубопровода,MOEK,2024-01-23 14:32:38.000000,NaN,2024-01-23 15:00:00.000000,2024-01-23 13:00:00.000000,2024-01-23 15:00:00.000000,Внеплановое,2933,внутригородская территория муниципальный округ...
70,Ремонт участка трубопровода,MOEK,2024-01-23 14:32:38.000000,NaN,2024-01-23 15:00:00.000000,2024-01-23 13:00:00.000000,2024-01-23 15:00:00.000000,Внеплановое,2936,внутригородская территория муниципальный округ...
71,Ремонт участка трубопровода,MOEK,2024-01-23 14:32:38.000000,NaN,2024-01-23 15:00:00.000000,2024-01-23 13:00:00.000000,2024-01-23 15:00:00.000000,Внеплановое,24147,внутригородская территория муниципальный округ...
72,Определение утечки,MOEK,2024-02-13 17:46:37.000000,NaN,2024-02-12 20:00:00.000000,2024-02-12 17:15:00.000000,2024-02-12 20:00:00.000000,Внеплановое,2105443,внутригородская территория муниципальный округ...
73,Определение утечки,MOEK,2024-02-13 17:46:37.000000,NaN,2024-02-12 20:00:00.000000,2024-02-12 17:15:00.000000,2024-02-12 20:00:00.000000,Внеплановое,19175,внутригородская территория муниципальный округ...


In [13]:
data_turn_of[data_turn_of['Причина'] == 'Замена регулятора']

,Причина,Источник,Дата регистрации отключения,Планируемая дата отключения,Планируемая дата включения,Фактическая дата отключения,Фактическая дата включения,Вид отключения,УНОМ,Адрес
74,Замена регулятора,MOEK,2024-04-04 12:23:38.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,1260,внутригородская территория муниципальный округ...
75,Замена регулятора,MOEK,2024-04-04 12:23:38.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,16883,внутригородская территория муниципальный округ...
76,Замена регулятора,MOEK,2024-04-04 12:26:02.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,16877,внутригородская территория муниципальный округ...
77,Замена регулятора,MOEK,2024-04-04 12:26:02.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,16875,внутригородская территория муниципальный округ...
78,Замена регулятора,MOEK,2024-04-04 12:26:02.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,16876,внутригородская территория муниципальный округ...
79,Замена регулятора,MOEK,2024-04-04 12:23:38.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,1258,внутригородская территория муниципальный округ...
80,Замена регулятора,MOEK,2024-04-04 12:23:38.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,16886,внутригородская территория муниципальный округ...
81,Замена регулятора,MOEK,2024-04-04 12:26:02.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,1259,внутригородская территория муниципальный округ...
82,Замена регулятора,MOEK,2024-04-04 12:26:02.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,2024-04-04 13:00:00.000000,2024-04-04 16:00:00.000000,Плановое,16885,внутригородская территория муниципальный округ...
96,Замена регулятора,MOEK,2024-04-18 11:40:41.000000,2024-04-18 14:00:00.000000,2024-04-18 18:00:00.000000,2024-04-18 14:00:00.000000,2024-04-18 18:00:00.000000,Плановое,28620,внутригородская территория муниципальный округ...


In [12]:
data_turn_of['Причина'].value_counts()

Причина
Установка приборов КИПиА       22
Снятие приборов учета          19
Замена запорной арматуры       19
Замена регулятора              17
Установка приборов учета       16
Ремонт ВВП                      7
Ремонт запорной арматуры        7
Определение утечки              5
Ремонт участка трубопровода     3
Name: count, dtype: int64

In [23]:
order_turn_off = data_turn_of[data_turn_of['Вид отключения'] == 'Плановое']
order_turn_off_date_dict = dict()
for i in range(order_turn_off.shape[0]):
    why = order_turn_off.iloc[i]['Причина']
    order_day_off = order_turn_off.iloc[i]['Планируемая дата отключения']
    order_day_on = order_turn_off.iloc[i]['Планируемая дата включения']
    fact_day_off = order_turn_off.iloc[i]['Фактическая дата отключения']
    fact_day_on = order_turn_off.iloc[i]['Фактическая дата включения']
    unom = order_turn_off.iloc[i]['УНОМ']
    order_turn_off_date_dict.setdefault(why, {
        'Планируемая дата включения': [],
        'Планируемая дата отключения': [],
        'Фактическая дата отключения': [],
        'Фактическая дата включения': [],
        'UNOM': []
    })
    order_turn_off_date_dict[why]['Планируемая дата включения'].append(order_day_on)
    order_turn_off_date_dict[why]['Планируемая дата отключения'].append(order_day_off)
    order_turn_off_date_dict[why]['Фактическая дата отключения'].append(fact_day_off)
    order_turn_off_date_dict[why]['Фактическая дата включения'].append(fact_day_on)
    order_turn_off_date_dict[why]['UNOM'].append(unom)

In [29]:
dfs = [pd.DataFrame(order_turn_off_date_dict[i]) for i in order_turn_off_date_dict]
for i, j in zip(dfs, order_turn_off_date_dict):
    i.name = j

In [33]:
def pa(a):
    print(a)
    return a

In [43]:
dates_dict = dict()
for i in dfs:
    dates_dict[i.name] = {
        "Планируемая дата отключения" : i.loc[:, "Планируемая дата отключения"].apply(lambda a: a[:10]).drop_duplicates().to_list(), 
        "Планируемая дата включения": i.loc[:, "Планируемая дата включения"].apply(lambda a: a[:10]).drop_duplicates().to_list()
    }

In [49]:
os.listdir('../main_datasets/timelines_datasets/')

['events_counts_time_lines.json']

In [50]:
with open('../main_datasets/timelines_datasets/turn_on_of_heat_ordered.json', 'w') as f:
    json.dump(dates_dict, f)

In [44]:
dates_dict

{'Установка приборов КИПиА': {'Планируемая дата отключения': ['2023-11-14',
   '2023-12-20',
   '2023-12-27',
   '2024-04-10'],
  'Планируемая дата включения': ['2023-11-14',
   '2023-12-20',
   '2023-12-27',
   '2024-04-10']},
 'Установка приборов учета': {'Планируемая дата отключения': ['2023-11-30',
   '2023-12-20'],
  'Планируемая дата включения': ['2023-11-30', '2023-12-20']},
 'Снятие приборов учета': {'Планируемая дата отключения': ['2023-10-11'],
  'Планируемая дата включения': ['2023-10-11']},
 'Замена запорной арматуры': {'Планируемая дата отключения': ['2023-12-22',
   '2024-04-26'],
  'Планируемая дата включения': ['2023-12-22', '2024-04-26']},
 'Ремонт ВВП': {'Планируемая дата отключения': ['2023-12-26'],
  'Планируемая дата включения': ['2023-12-26']},
 'Замена регулятора': {'Планируемая дата отключения': ['2024-04-04',
   '2024-04-18'],
  'Планируемая дата включения': ['2024-04-04', '2024-04-18']},
 'Ремонт запорной арматуры': {'Планируемая дата отключения': ['2024-04-10